In [ ]:
from analysis_utils import merge_csv_files
year = 2024
csv_dir_path = f'output/공고별_기업_투찰정보_{year}'
df = merge_csv_files(csv_dir_path)
len(df)

In [ ]:
# extract 0.46319 from 0.46319 (100.46319)
df['기초대비사정률'] = df['기초대비 사정률(%)'].str.extract(r'(-?\d+\.\d+)')[0].astype(float)

# 1. 먼저 사업자별로 그룹화하고 기본 통계량 계산
company_stats = df.groupby('사업자 등록번호').agg({
    '기초대비사정률': ['count', 'mean', 'std', 'min', 'max']
}).round(4) 

# 2. 참여 횟수로 필터링 (예: 10회 이상 참여한 업체만)
min_participations = 10
filtered_stats = company_stats[company_stats[('기초대비사정률', 'count')] >= min_participations]

# 3. 통계량 분포 시각화
fig, axes = plt.subplots(2, 2, figsize=(15, 10))

# 평균 분포
axes[0,0].hist(filtered_stats[('기초대비사정률', 'mean')], bins=50)
axes[0,0].set_title('Distribution of Mean 기초대비사정률 by Company')
axes[0,0].set_xlabel('Mean')
axes[0,0].set_xlim(0, 2)
axes[0,0].set_ylabel('Frequency')

# 표준편차 분포
axes[0,1].hist(filtered_stats[('기초대비사정률', 'std')], bins=50)
axes[0,1].set_title('Distribution of Std 기초대비사정률 by Company')
axes[0,1].set_xlabel('Standard Deviation')
axes[0,1].set_ylabel('Frequency')

# 참여 횟수 분포
axes[1,0].hist(filtered_stats[('기초대비사정률', 'count')], bins=50)
axes[1,0].set_title('Distribution of Participation Count by Company')
axes[1,0].set_xlabel('Number of Participations')
axes[1,0].set_ylabel('Frequency')

# 평균 vs 표준편차 산점도
axes[1,1].scatter(filtered_stats[('기초대비사정률', 'mean')], 
                 filtered_stats[('기초대비사정률', 'std')], 
                 alpha=0.5)
axes[1,1].set_title('Mean vs Std of 기초대비사정률')
axes[1,1].set_xlabel('Mean')
axes[1,1].set_ylabel('Standard Deviation')

plt.tight_layout()
plt.show()

# 4. 상위/하위 통계량 출력
print("\n=== 평균이 가장 높은 업체 Top 10 ===")
print(filtered_stats.nlargest(10, ('기초대비사정률', 'mean')))

print("\n=== 표준편차가 가장 작은 업체 Top 10 ===")
print(filtered_stats.nsmallest(10, ('기초대비사정률', 'std')))

print("\n=== 참여횟수가 가장 많은 업체 Top 10 ===")
print(filtered_stats.nlargest(10, ('기초대비사정률', 'count')))

In [ ]:
# print("\n=== 참여횟수가 가장 많은 업체 Top 10 ===")
# print(filtered_stats.nlargest(10, ('기초대비사정률', 'count')))

# 표준편차 분포의 평균을 구한다.


# 표준편차의 평균 계산
std_mean = filtered_stats[('기초대비사정률', 'std')].mean()
print(f"표준편차의 평균: {std_mean:.4f}")

# 추가로 표준편차의 중앙값도 확인하면 유용할 수 있습니다
std_median = filtered_stats[('기초대비사정률', 'std')].median()
print(f"표준편차의 중앙값: {std_median:.4f}")

In [ ]:
# 1000 이하의 데이터만 필터링
filtered_std = filtered_stats[filtered_stats[('기초대비사정률', 'std')] <= 10]

# 필터링된 데이터의 기술통계량 계산
std_stats = filtered_std[('기초대비사정률', 'std')].describe()
print("=== 표준편차의 기술통계량 (1000 이하) ===")
print(f"개수: {std_stats['count']:.0f}")
print(f"평균: {std_stats['mean']:.4f}")
print(f"표준편차: {std_stats['std']:.4f}")
print(f"최소값: {std_stats['min']:.4f}")
print(f"25%: {std_stats['25%']:.4f}")
print(f"50%(중앙값): {std_stats['50%']:.4f}")
print(f"75%: {std_stats['75%']:.4f}")
print(f"최대값: {std_stats['max']:.4f}")

# Boxplot 생성
plt.figure(figsize=(10, 6))
plt.boxplot(filtered_std[('기초대비사정률', 'std')], whis=1.5)
plt.title('Distribution of Standard Deviations by Company (≤ 1000)')
plt.ylabel('Standard Deviation')
plt.grid(True, alpha=0.3)
plt.show()

# 제외된 데이터 수 출력
excluded_count = len(filtered_stats) - len(filtered_std)
print(f"\n1000을 초과하는 데이터 수: {excluded_count}")

In [ ]:
import pandas as pd
pd.set_option('display.max_rows', None)  # 모든 행 표시
pd.set_option('display.max_columns', None)  # 모든 열 표시
pd.set_option('display.width', None)  # 너비 제한 해제
pd.set_option('display.max_colwidth', None)  # 컬럼 너비 제한 해제

print("\n=== 표준편차가 가장 작은 업체 Top 10 ===")
print(filtered_stats.nsmallest(100, ('기초대비사정률', 'std')))

In [ ]:
# 1000 이하의 데이터만 필터링
filtered_std = filtered_stats[filtered_stats[('기초대비사정률', 'std')] <= 10]
# 필터링된 데이터의 기술통계량 계산
std_stats = filtered_std[('기초대비사정률', 'std')].describe()
print("=== 표준편차의 기술통계량 (1000 이하) ===")
print(f"개수: {std_stats['count']:.0f}")
print(f"평균: {std_stats['mean']:.4f}")
print(f"표준편차: {std_stats['std']:.4f}")
print(f"최소값: {std_stats['min']:.4f}")
print(f"25%: {std_stats['25%']:.4f}")
print(f"50%(중앙값): {std_stats['50%']:.4f}")
print(f"75%: {std_stats['75%']:.4f}")
print(f"최대값: {std_stats['max']:.4f}")

# Boxplot 생성
plt.figure(figsize=(10, 6))
plt.boxplot(filtered_std[('기초대비사정률', 'std')], whis=1.5)
plt.title('Distribution of Standard Deviations by Company (≤ 1000)')
plt.ylabel('Standard Deviation')
plt.grid(True, alpha=0.3)
plt.show()

# 제외된 데이터 수 출력
excluded_count = len(filtered_stats) - len(filtered_std)
print(f"\n1000을 초과하는 데이터 수: {excluded_count}")

In [ ]:
# 3. 통계량 분포 시각화
fig, axes = plt.subplots(1, 1, figsize=(15, 10))

# 표준편차 분포
axes.hist(filtered_stats[('기초대비사정률', 'std')], bins=50000)
axes.set_title('Distribution of d 기초대비사정률 by Company')
axes.set_xlabel('Standard DeviatiSton')
axes.set_ylabel('Frequency')
axes.set_xlim(0,2)
plt.show()
